In [ ]:
import csv
import json
# from datetime import datetime, timezone
# from dateutil import parser

In [ ]:
csvfile = open('discord_chat.csv', 'r')
jsonfile = open('dataset.jsonl', 'w')

fieldnames = ("AuthorID","Author","Date","Content","Attachments","Reactions")
reader = csv.DictReader( csvfile, fieldnames)

In [ ]:
# find pairs of lines where person A speaks, then person B speaks
# append them as two separate lines in rows_to_keep
print("Parsing data...")
rows_to_keep = []
last_row = None
for row in reader:
    # if parser.isoparse(row["Date"]) > datetime(2024, 1, 6, tzinfo=timezone.utc) and parser.isoparse(row["Date"]) < datetime(2024, 1, 10, tzinfo=timezone.utc):
    if last_row is not None and row is not None:
        if row["Author"] != last_row["Author"]:
            rows_to_keep.append(last_row)
            last_row = row
        else:
            last_row["Content"] = last_row["Content"] + " " + row["Content"]
    else:
        last_row = row

In [ ]:
# concatenate data
prompts = []
responses = []
for i in range(2, len(rows_to_keep), 2):
    if i+1 < len(rows_to_keep):
        prompt = rows_to_keep[i]
        response = rows_to_keep[i+1]
        prompt = prompt["Content"].replace('"','\\"')
        response = response["Content"].replace('"','\\"')

        prompts.append(prompt)
        responses.append(response)

In [ ]:
# convert data to jsonl
for i in range(len(prompts)):
    fields = {
        "input": prompts[i], 
        "response": responses[i]
    }
    json.dump(fields, jsonfile)
    jsonfile.write('\n')